In [1]:
import pandas
import os

In [2]:
import utils

In [22]:
import os
from math import radians, cos, sin, asin, sqrt
import sys
import time

import sys
sys.path.append('/Users/gustaveronteix/Documents/Projets/Projets Code/batch7_beges/datasets/chorus-dt')

import requests
import dill
import urllib.parse
import pandas as pd
from tqdm import tqdm
from utils.config import CONFIG

from utils import dataset
from utils.resolvers import HardcodesResolver, GeocodingApiResolver

tqdm.pandas()


def compute_distance(x):
    lon1 = x["coords_place_0_lon"]
    lat1 = x["coords_place_0_lat"]
    lon2 = x["coords_place_1_lon"]
    lat2 = x["coords_place_1_lat"]
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in miles. Use 6371 for kilometers
    return c * r


if __name__ == "__main__":
    data = dataset.load_data("./data/raw/sample-avion-train.csv")
    places, trips = dataset.get_places_and_trips(data, ["T", "TC", "TCA", "TCM", "TU"])

    # Splitting codes / name
    places[["code_1", "name", "temp", "code_2"]] = places["place"].str.extract(
        "([^-]*)\s-\s([^\[]*)\s?([\[](.*)[\]])?", expand=True
    )
    places.drop(columns=["temp"], inplace=True)

    places["resolved"] = False
    places["resolved_through_uic_code"] = False
    places["resolved_through_insee_code"] = False
    places["resolved_through_tvs_code"] = False
    places["resolved_through_iata_code"] = False
    places["resolved_through_gmap_api"] = False
    places["lon"] = None
    places["address"] = ""
    places["lat"] = None

    hardcode_resolver = HardcodesResolver()
    places = places.progress_apply(hardcode_resolver.resolve, axis=1)

    # Initiate geocoder
    GMAP_API_KEY = CONFIG["api_keys"]["gmap"]
    geocoding_api_resolver = GeocodingApiResolver(api_name="gmap", api_key=GMAP_API_KEY)

    # Apply the geocoder with delay using the rate limiter:
    places.loc[~places["resolved"], :] = places[~places["resolved"]].progress_apply(
        geocoding_api_resolver.resolve, axis=1
    )
    geocoding_api_resolver.save_cache()
    for c in places:
        if "resolved_through_" in c:
            print(c, places[c].sum())

    places["lat"] = places["lat"].astype(float)
    places["lon"] = places["lon"].astype(float)
    places_dict = places.set_index("place").to_dict()

    for place_index in [0, 1, 2]:
        trips["coords_place_%d_lat" % place_index] = trips[
            "trip_place_%d" % place_index
        ].apply(lambda x: places_dict["lat"][x])
        trips["coords_place_%d_lon" % place_index] = trips[
            "trip_place_%d" % place_index
        ].apply(lambda x: places_dict["lon"][x])
        trips["coords_place_%d" % place_index] = (
            trips["coords_place_%d_lon" % place_index].astype(str)
            + ";"
            + trips["coords_place_%d_lat" % place_index].astype(str)
        )
        trips["place_%d_count" % place_index] = trips[
            "trip_place_%d" % place_index
        ].apply(lambda x: places_dict["total"][x])

    trips["distance"] = trips.apply(compute_distance, axis=1)
    places.to_csv("./data/clean/places.csv", index=False)
    trips.to_csv("./data/clean/trips.csv", index=False)


/Users/gustaveronteix/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Prestation types: 
['T' 'TU' 'TCA' 'TC']
Unique places src:  1009
Unique places dst:  1001
Unique places stop:  1409
Unique places:  1668
Unique trips:  16574


100%|██████████| 1668/1668 [00:00<00:00, 3370.75it/s]

nan



100%|██████████| 570/570 [00:00<00:00, 1677.46it/s]


resolved_through_uic_code 98
resolved_through_insee_code 884
resolved_through_tvs_code 0
resolved_through_iata_code 116
resolved_through_gmap_api 567


In [24]:
places[places['resolved_through_tvs_code']]

,place,src,dst,stop,total,code_1,name,code_2,resolved,resolved_through_uic_code,resolved_through_insee_code,resolved_through_tvs_code,resolved_through_iata_code,resolved_through_gmap_api,lon,address,lat


In [ ]:
import numpy as np

data = dataset.load_data("./data/raw/sample-avion-train.csv")
places, trips = dataset.get_places_and_trips(data, ["T", "TC", "TCA", "TCM", "TU"])

In [ ]:
places

In [ ]:
places[places['resolved_through_iata_code']]['name'].str.contains('orly')

In [26]:
if __name__ == "__main__":
    data = dataset.load_data("./data/raw/sample-avion-train.csv")
    places, trips = dataset.get_places_and_trips(data, ["T", "TC", "TCA", "TCM", "TU"])

    # Splitting codes / name
    places[["code_1", "name", "temp", "code_2"]] = places["place"].str.extract(
        "([^-]*)\s-\s([^\[]*)\s?([\[](.*)[\]])?", expand=True
    )
    places.drop(columns=["temp"], inplace=True)

    places["resolved"] = False
    places["resolved_through_uic_code"] = False
    places["resolved_through_insee_code"] = False
    places["resolved_through_tvs_code"] = False
    places["resolved_through_iata_code"] = False
    places["resolved_through_gmap_api"] = False
    places["lon"] = None
    places["address"] = ""
    places["lat"] = None

    hardcode_resolver = HardcodesResolver()
    places = places.progress_apply(hardcode_resolver.resolve, axis=1)

Prestation types: 
['T' 'TU' 'TCA' 'TC']
Unique places src:  1009
Unique places dst:  1001
Unique places stop:  1409
Unique places:  1668
Unique trips:  16574


 33%|███▎      | 548/1668 [00:00<00:00, 2579.61it/s]

TR_FRPNO
TR_FRPNO
75056
TR_FRPLY
TR_FRPMO
TR_BEBMI
TR_FRLPD
TR_FRPST
TR_FRLAB
TR_FRBOJ

TR_FRNTE
IA_PAR
TR_FRAFK
TR_FRMSC
TR_FRRNS

TR_FRPAZ
TR_FRADE
TR_FRADJ
TR_FRPSL
TR_FRAIE
69123
TR_FRXSB
TR_FRMPL


TR_FRLLE
13055
TR_FRURD
DEP075
TR_FRAEG
TR_FRLSC
TR_FRACL
TR_FRVLA
TR_FRABA
TR_FRXSH

TR_FRQRV

TR_FRAFW
TR_FRENC
TR_FRCFR
TR_NLRTA
TR_CHGVA
TR_FRAVG
TR_FRXYT

TR_FRLPE


TR_FRJDQ
TR_FRGNB
TR_FRFNI

TR_FRXNA


TR_FRDXK
TR_LULUX
TR_FRHVS
TR_FRXVS
TR_FRPIS

TR_FRAET
TR_FRACK
TR_FRBES
IA_QXB
TR_FRAEZ
TR_FRTLN

TR_FRPBE
TR_FRADI


TR_FRLCH
TR_FRRHE


TR_FRAES
TR_FRSTH
TR_FRQAM

TR_FRADP

TR_FRLRH
TR_NLQRH

TR_FRTJB

TR_FRUIP
TR_FRMZN
92055
TR_FRMLV
TR_FRACN

TR_FRCFE
TR_FRDOL
TR_FRVNE



TR_FREAH
TR_FRFRK


TR_FRXLE
TR_FRXDN

TR_FRHHD
TR_CHBRN
TR_FRBTB
IA_LON


TR_FRXAS
IA_XSH
TR_FRTQY
TR_FRXSY
TR_FRPLY
TR_FRATW
TR_FRTLG
IA_QLS
TR_FRCMF
TR_FRGGP
TR_FRAEK
TR_FRSNR
TR_FRXBQ
TR_FRXAI
TR_FRABG
TR_FRCCF
TR_FRDTC
TR_FRXBF
TR_FRXUY
TR_FRAFD
TR_FRNIT
TR_FRMLT
TR_FRAEJ
DEP075
TR_FRLRT
TR_FRNVS
TR_FR

 68%|██████▊   | 1140/1668 [00:00<00:00, 2769.24it/s]


TR_DEDUI
TR_FRXSJ
TR_FRXSW

TR_FRDTO
TR_FRADM
TR_FRMSC
TR_FRACK
IA_QAM



TR_FRHLF
IA_ZMU
TR_FRTLN
TR_FRXST
TR_FRAMB
IA_ZGP
TR_FRCYX




TR_FRCIJ
TR_FRACR

TR_FRMZQ
TR_FRHZJ
TR_FRMCX
TR_FRYVE
IA_XSS

TR_FRIEO

TR_FRHOV
TR_FRHFB

TR_FRACG
TR_FRXTS
TR_FRCDW
TR_FREEU
TR_FRGYB


IA_ZLN
TR_FRHZK
TR_FRGWE
IA_XBK

IA_QFB

TR_FROIC
TR_FRHWB



TR_FRRYN
IA_XSK
TR_FREMZ
IA_XBY
TR_BELGG
IA_ZJP
IA_XLE
TR_FRCET
TR_FRQXB

TR_CHZJP
TR_FRCJD
TR_FRDXV

TR_FRILV
IA_ZBC



TR_NLEIN
IA_QEV


TR_FRMEN

TR_FRBGJ
TR_BEOST
TR_FRTOL
TR_FRXBV
TR_FRLAM
TR_FREJU

IA_XAC
TR_FRXAG
TR_FRINC
TR_FRXGF
TR_FRBWD
TR_FRSPZ
TR_FRFET
TR_FRGAT


TR_FRXSH
TR_FRRNE

TR_FRLPE
TR_FRNIC
TR_FRBYX
TR_FRHWE
IA_RSY

TR_FRMPV
TR_FRXCX

TR_FRJCA
TR_FREVX
TR_FRHTW
TR_FRCMF


TR_FRAEG

TR_FREPJ

IA_XUY

TR_FRHCT

TR_FRBJJ
TR_FRVFP


TR_FRAPI
TR_FRCRW

TR_FRDUX

TR_FRXAT
TR_FRHHQ
IA_XDN

IA_JPU

IA_QRV
IA_XCD
TR_FRRIO
TR_FRXFX



TR_NLSPL

TR_FRABA
IA_XGB
TR_FRBIQ
TR_FRCCY
TR_FRAAW
TR_FRXCW
TR_FRPMO

TR_FRAUR
TR_FRAPM
IA_XVZ

TR_FRVAF
TR

100%|██████████| 1668/1668 [00:00<00:00, 2795.72it/s]

DEP094
IA_XAS
38024

IA_XCP
IA_XCK
22007
IB_GRAN
IA_QNL
IA_QYR
92055
TR_FRLAB
CHIU
TR_FREAH
IB_OSN
IA_XRF
TR_FRXYT
78524
IB_COUR

IA_XSG
IA_XAT
IA_XSJ
IA_QJZ



IA_MLH


IA_ZNV
IA_XCY

IA_QBQ
IB_TURI

78251


IA_QKU

IA_XPJ

IA_NYC


IA_XBD
TR_FRLLE



In [28]:
places[places['resolved_through_tvs_code']]

,place,src,dst,stop,total,code_1,name,code_2,resolved,resolved_through_uic_code,resolved_through_insee_code,resolved_through_tvs_code,resolved_through_iata_code,resolved_through_gmap_api,lon,address,lat


In [29]:
a = places.loc[0]
with open(("./data/prepared/tvs_codes.pkl"), "rb") as file_id:
    tvs_codes = dill.load(file_id)

In [31]:
a["code_1"]

'TR_FRPNO'

In [32]:
a["code_1"] in tvs_codes['Longitude WGS84']

True

In [33]:
a["lon"] = tvs_codes['Longitude WGS84'][a["code_1"]]
a["lat"] = tvs_codes['Latitude WGS84'][a["code_1"]]

/Users/gustaveronteix/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/gustaveronteix/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
a

place                          TR_FRPNO - Paris Nord
src                                             3212
dst                                             3190
stop                                             194
total                                           6596
code_1                                      TR_FRPNO
name                                      Paris Nord
code_2                                           NaN
resolved                                       False
resolved_through_uic_code                      False
resolved_through_insee_code                    False
resolved_through_tvs_code                      False
resolved_through_iata_code                     False
resolved_through_gmap_api                      False
lon                                          2.35515
address                                             
lat                                          48.8802
Name: 0, dtype: object

In [17]:
hardcode_resolver = HardcodesResolver()
tvs_codes = hardcode_resolver.tvs_codes

In [21]:
tvs_codes['Longitude WGS84']['TR_FRPNO']

2.3551509999999998

In [25]:
class HardcodesResolver:
    def __init__(self):
        self.insee_codes = self.load_codes("./data/prepared/insee_codes.pkl")
        self.tvs_codes = self.load_codes("./data/prepared/tvs_codes.pkl")
        self.uic_codes = self.load_codes("./data/prepared/uic_codes.pkl")
        self.iata_codes = self.load_codes("./data/prepared/iata_codes.pkl")

    def load_codes(self, path):
        with open(path, "rb") as file_id:
            return dill.load(file_id)

    def resolve(self, x):
        code_2_as_int = self.get_int(x["code_2"])
        
        try:
            if code_2_as_int and (code_2_as_int in self.uic_codes["lon"]):
                x["lon"] = self.uic_codes["lon"][code_2_as_int]
                x["lat"] = self.uic_codes["lat"][code_2_as_int]
                x["resolved"] = True
                x["resolved_through_uic_code"] = True
            elif x["code_1"] in self.insee_codes["lon"]:
                x["lon"] = self.insee_codes["lon"][x["code_1"]]
                x["lat"] = self.insee_codes["lat"][x["code_1"]]
                x["resolved"] = True
                x["resolved_through_insee_code"] = True
            elif ((str(x["code_1"][-3:]) in self.iata_codes["lon"]) and
                 (str(x["code_1"][:2]) == 'IA')):                
                x["lon"] = self.iata_codes["lon"][x["code_1"][-3:]]
                x["lat"] = self.iata_codes["lat"][x["code_1"][-3:]]
                x["resolved"] = True
                x["resolved_through_iata_code"] = True
            elif x["place"] == '-75056': # Hard exception...
                x["name"] = 'Paris'
            elif x["name"].contains('Orly'):
                x["lon"] = 2.39468296999
                x["lat"] = 2.39468296999
            elif x['code_1'] in self.tvs_codes['Longitude WGS84']:
                x['lon'] = self.tvs_codes['Longitude WGS84'][x['code_1']]
                x['lat'] = self.tvs_codes['Latitude WGS84'][x['code_1']]
                x['resolved'] = True
                x['resolved_through_tvs_code'] = True
            else:
                x["lon"] = None
                x["lat"] = None
                
        except:
            print(x["code_1"])
        return x

    def get_int(self, x):
        try:
            return int(x)
        except ValueError:
            return None